<a href="https://colab.research.google.com/github/SOMBIEB/dodge-game-ppo-qlearning/blob/main/Projetencoursgeneration1video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Création d'un mini jeu d'esquive** dans le cadre de mon projet de fin de module de programmation par renforcement. Le but du projet c'est d'entrainer un agent IA a esquivé  les obstacles en allant de gauche à droite et au milieu dans un environnement simple que j'ai crér avec Pygame.


**Implémentation du mini-jeu en PyGame**
Le script PyGame suivant permet à un utilisateur humain de jouer à une version du jeu Dodge où il contrôle un carré vert pour éviter des obstacles rouges. Cette version est  la version jouable manuellement avec les touches gauche et droite.


In [1]:
import pygame
import random
import sys

# Initialisation
pygame.init()
WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

# Joueur
player = pygame.Rect(WIDTH // 2 - 20, HEIGHT - 50, 40, 40)
player_speed = 7

# Obstacles
obstacles = []
for _ in range(3):  # 3 obstacles au début
    x = random.randint(0, WIDTH - 40)
    obstacles.append(pygame.Rect(x, -40, 40, 40))

obstacle_speed = 3
score = 0
level = 1

# Texte
font = pygame.font.SysFont(None, 30)

def draw_game():
    screen.fill((20, 20, 20))
    pygame.draw.rect(screen, (0, 255, 0), player)
    for obs in obstacles:
        pygame.draw.rect(screen, (255, 0, 0), obs)

    # Score et niveau
    text = font.render(f"Score : {score}   Niveau : {level}", True, (255, 255, 255))
    screen.blit(text, (10, 10))

    pygame.display.flip()

# Boucle principale
running = True
while running:
    draw_game()
    clock.tick(30)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Contrôle clavier
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] and player.left > 0:
        player.move_ip(-player_speed, 0)
    if keys[pygame.K_RIGHT] and player.right < WIDTH:
        player.move_ip(player_speed, 0)

    # Mise à jour des obstacles
    for i in range(len(obstacles)):
        obstacles[i].move_ip(0, obstacle_speed)
        if obstacles[i].top > HEIGHT:
            score += 1
            x = random.randint(0, WIDTH - 40)
            obstacles[i] = pygame.Rect(x, -40, 40, 40)

    # Collision
    for obs in obstacles:
        if player.colliderect(obs):
            print(" Collision !")
            running = False

    # Niveau suivant chaque 10 points
    if score > 0 and score % 10 == 0:
        level += 1
        obstacle_speed += 1
        score += 1  # éviter de répéter le changement de niveau


pygame 2.6.1 (SDL 2.28.4, Python 3.11.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
 Collision !


In [2]:
!pip install stable-baselines3[extra] gym pygame



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

**Définition de l’environnement Gymnasium personnalisé (DodgeGameEnv):** Cette classe définit un environnement compatible avec Gymnasium, représentant une version simplifiée du jeu de mon jeu.
Le joueur (carré vert) peut se déplacer horizontalement sur 3 positions pour éviter un obstacle (carré rouge) qui apparaît à une position aléatoire.


In [11]:
import gymnasium as gym
import numpy as np
import pygame

class DodgeGameEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 30}

    def __init__(self, render_mode="rgb_array"):
        super().__init__()
        self.render_mode = render_mode
        self.window_size = 300
        self.observation_space = gym.spaces.Box(low=0, high=2, shape=(2,), dtype=np.int32)
        self.action_space = gym.spaces.Discrete(3)  # 0: Gauche, 1: Rien, 2: Droite

        self.player_pos = 1
        self.obstacle_pos = 1
        self.timestep = 0
        self.clock = None
        self.screen = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.player_pos = 1
        self.obstacle_pos = np.random.randint(0, 3)
        self.timestep = 0  # compteur de temps
        obs = np.array([self.obstacle_pos, self.player_pos], dtype=np.int32)
        return obs, {}

    def step(self, action):
        if action == 0:
            self.player_pos = max(0, self.player_pos - 1)
        elif action == 2:
            self.player_pos = min(2, self.player_pos + 1)

        reward = 1 if self.player_pos != self.obstacle_pos else -1

        self.timestep += 1
        terminated = self.timestep >= 20  # L'épisode dure 20 steps
        truncated = False

        obs = np.array([self.obstacle_pos, self.player_pos], dtype=np.int32)
        info = {}
        return obs, reward, terminated, truncated, info

    def render(self):
        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((self.window_size, self.window_size))
            self.clock = pygame.time.Clock()

        self.screen.fill((0, 0, 0))
        cell_size = self.window_size // 3

        # Affichage
        pygame.draw.rect(self.screen, (255, 0, 0), pygame.Rect(self.obstacle_pos * cell_size, 50, cell_size, cell_size))  # obstacle
        pygame.draw.rect(self.screen, (0, 255, 0), pygame.Rect(self.player_pos * cell_size, 200, cell_size, cell_size))  # joueur

        if self.render_mode == "human":
            pygame.display.flip()
            self.clock.tick(self.metadata["render_fps"])
        elif self.render_mode == "rgb_array":
            return pygame.surfarray.array3d(pygame.display.get_surface())

    def close(self):
        if self.screen is not None:
            pygame.quit()
            self.screen = None


In [12]:
!pip install gymnasium stable-baselines3[extra]


**Vérification de la compatibilité de l’environnement avec Gymnasium**

In [13]:
from stable_baselines3.common.env_checker import check_env
env = DodgeGameEnv()
check_env(env)


 **Entraînement de l’agent avec PPO (Proximal Policy Optimization)** : Dans cette section, nous utilisons `stable-baselines3` pour entraîner un agent basé sur la méthode PPO comme indiqué dans les consignes. Les etapes importante de cette parties conistent a definir l'environnement vectorisé avec `DummyVecEnv`, a utiliser une  politique de type `MlpPolicy`,et a l'entraînement sur `100 000 étapes` avec affichage `verbose=1` pour suivre le processus.


In [14]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Environnement vectorisé
env = DummyVecEnv([lambda: DodgeGameEnv()])

# Définir l'agent
model = PPO("MlpPolicy", env, verbose=1)

# Entraînement
model.learn(total_timesteps=100_000)


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 409  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 456         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013550483 |
|    clip_fraction        | 0.212       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.00823    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.05        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0305     |
|    value_loss           | 12.8        |
-----------------------------------------
-----------------

In [19]:
model.save("ppo-dodge-agent")


**Génération d’une vidéo automatique de l’agent (PPO):** Ce bloc permet d'enregistrer une démonstration visuelle de l’agent PPO entraîné en action, dans l’environnement `DodgeGameEnv`.

In [27]:
from stable_baselines3.common.vec_env import VecVideoRecorder

# Recrée l'environnement en mode vidéo
env = DummyVecEnv([lambda: DodgeGameEnv(render_mode="rgb_array")])
env = VecVideoRecorder(env, "./videos/",
                       record_video_trigger=lambda x: x == 0,
                       video_length=100,
                       name_prefix="dodge_agent")

obs = env.reset()
for _ in range(100):
    action, _ = model.predict(obs)
    obs, _, done, _ = env.step(action)

env.close()


Saving video to /content/dodge-game-ppo-qlearning/videos/dodge_agent-step-0-to-step-100.mp4
Moviepy - Building video /content/dodge-game-ppo-qlearning/videos/dodge_agent-step-0-to-step-100.mp4.
Moviepy - Writing video /content/dodge-game-ppo-qlearning/videos/dodge_agent-step-0-to-step-100.mp4



Moviepy - Done !
Moviepy - video ready /content/dodge-game-ppo-qlearning/videos/dodge_agent-step-0-to-step-100.mp4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
obs = env.reset()
for i in range(50):
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()


cette boucle permet de **le faire jouer automatiquement** L'Agent dans l’environnement pendant 100 étapes.

Deuxieme Technique :Implémentation du Q-learning
Les parties importantes de cette session sont l'entraînement avec Q-table, l'exploration, et lamise à jour des valeurs.

In [16]:
import numpy as np
import matplotlib.pyplot as plt

# initialisation
n_states = 3  # positions possibles du joueur
n_actions = 3
q_table = np.zeros((n_states, n_actions))

alpha = 0.1
gamma = 0.9
epsilon = 0.1
n_episodes = 500
max_steps = 20

rewards = []

#  Fonction de récompense
def get_reward(player, obstacle):
    return 1 if player != obstacle else -1

#  Entraînement
for episode in range(n_episodes):
    player_pos = 1
    obstacle_pos = np.random.randint(0, 3)
    total_reward = 0

    for _ in range(max_steps):
        # Choix de l'action (exploration ou exploitation)
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.choice(n_actions)
        else:
            action = np.argmax(q_table[player_pos])

        # Appliquer l'action
        if action == 0:
            player_pos = max(0, player_pos - 1)
        elif action == 2:
            player_pos = min(2, player_pos + 1)

        reward = get_reward(player_pos, obstacle_pos)
        total_reward += reward

        # Q-learning initiation
        next_state = player_pos
        q_table[player_pos, action] += alpha * (
            reward + gamma * np.max(q_table[next_state]) - q_table[player_pos, action]
        )

    rewards.append(total_reward)


print("Q-table apprise :")
print(q_table)


Q-table apprise :
[[2.87346315 0.850522   0.        ]
 [4.6616459  3.79902855 4.85571161]
 [0.         2.09985886 3.32428775]]


In [20]:
!pip install -q huggingface_hub


In [21]:
from huggingface_hub import notebook_login
notebook_login()


In [25]:
!git config --global user.email "sombiebibata99@gmail.com"
!git config --global user.name "SOMBIEB"
